In [59]:
from read_api_keys import read_api_keys
import itertools
import numpy as np
import pandas as pd
import time
import sys
sys.path.append('../')
from pybit_ms.bybit_client import BybitAPI

In [12]:
keys = read_api_keys("TAK.txt")
public_key = keys.get('PUBLIC_KEY')
private_key = keys.get('PRIVATE_KEY')


client = BybitAPI(api_key=public_key, api_secret=private_key, testnet=1)
client

BybitAPI(testnet=1)

In [13]:
COINS = ["BTC", "ETH", "BNB", "SOL", "XRP", "DOGE", "ADA", "LINK", "DOT", "TRX"]
COIN_PAIRS = list(itertools.combinations(COINS, 2))

In [24]:
data = pd.DataFrame()

for coin in COINS:
    # Get Kline of previous 30 days with interval of 1 hour
    data[f'{coin}'] = client.market.get_kline(category="linear", coin1=coin, coin2="USDT", interval="60", price_type="close", limit=720)
    
    # Convert prices to float
    data[f'{coin}'] = pd.to_numeric(data[f'{coin}'], errors='coerce')
    
    # Normalize the prices
    min_price = data[f'{coin}'].min()
    max_price = data[f'{coin}'].max()
    data[f'{coin}'] = (data[f'{coin}']) - min_price / (max_price - min_price)
    
data

,BTC,ETH,BNB,SOL,XRP,DOGE,ADA,LINK,DOT,TRX
0,32423.099128,3505.658416,703.226044,180.566583,0.348525,-1.121647,-0.796462,17.550654,5.2733,-1.433901
1,39359.999128,3288.858416,684.726044,195.486583,0.153225,-1.148317,-0.793362,18.953654,5.0613,-1.426961
2,47974.699128,3253.798416,688.226044,196.096583,0.161125,-1.109017,-0.724762,20.661654,4.9893,-1.412331
3,51821.999128,3186.058416,692.726044,184.286583,0.229125,-1.155297,-0.799162,18.220654,4.5673,-1.428281
4,59869.399128,3156.328416,698.226044,179.586583,0.014225,-1.132627,-0.779262,17.124654,4.8573,-1.416821
...,...,...,...,...,...,...,...,...,...,...
715,74330.499128,3559.718416,678.226044,212.676583,0.340225,-1.042357,-0.601462,20.166654,6.7593,-1.393181
716,74519.899128,3599.958416,656.726044,210.946583,0.291025,-1.071647,-0.682662,20.634654,6.4403,-1.386811
717,78529.499128,3546.548416,669.726044,214.386583,0.148925,-1.071647,-0.681562,20.886654,6.7833,-1.379761
718,74330.799128,3600.168416,643.726044,205.286583,0.056325,-1.065587,-0.664062,19.764654,6.6623,-1.382141


In [ ]:
!pip install statsmodels

In [44]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

def calc_adf(data, coin1, coin2):
    Y = data[f'{coin1}']
    X = data[f'{coin2}']
    X = sm.add_constant(X)  
    model = sm.OLS(Y, X).fit()
    residuals = model.resid
    result = adfuller(residuals)[0]
    return result

# Find pairs with highest cointegration
def coint_pairs(data):  
    adfs = {}
    for pair in COIN_PAIRS:
        coin1 = pair[0]
        coin2 = pair[1]
        adfs[f"{coin1}-{coin2}"] = calc_adf(data, coin1, coin2)
    adfs = dict(sorted(adfs.items(), key=lambda item: item[1], reverse=True))
    return list(adfs.items())

top_pair = coint_pairs(data)[:1][0]
print(top_pair)
coin1 = top_pair[0][:top_pair[0].index('-')]
coin2 = top_pair[0][top_pair[0].index('-')+1:]


('DOGE-LINK', np.float64(-1.1230586236417373))


In [58]:
q = 1000
open_position=False

while True:
	coin1_past_5days = client.market.get_kline(category="linear", coin1=coin1, coin2="USDT", interval="60", price_type="close", limit=120)
	coin2_past_5days = client.market.get_kline(category="linear", coin1=coin2, coin2="USDT", interval="60", price_type="close", limit=120)
	
	coin1_past_5days = np.array(coin1_past_5days, dtype=float)
	coin2_past_5days = np.array(coin2_past_5days, dtype=float)
	
	min_price1 = coin1_past_5days.min()
	max_price1 = coin1_past_5days.max()
	coin1_past_5days = (coin1_past_5days - min_price1) / (max_price1 - min_price1)
	
	min_price2 = coin2_past_5days.min()
	max_price2 = coin2_past_5days.max()
	coin2_past_5days = (coin2_past_5days - min_price2) / (max_price2 - min_price2)
	
	spread = coin2_past_5days - coin1_past_5days
	spread_mean = spread.mean()
	spread_std = spread.std()
	
	up_band = spread_mean + 2 * spread_std
	low_band = spread_mean - 2 * spread_std

	print("Calculated stats...")
	

	coin1_price = client.market.get_tickers(category="linear", symbol=f"{coin1}USDT", only_ticker=True)
	coin2_price = client.market.get_tickers(category="linear", symbol=f"{coin2}USDT", only_ticker=True)

	
	if spread[-1] < low_band and not open_position:
		client.trade.place_futures_order(symbol=f"{coin1}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
		client.trade.place_futures_order(symbol=f"{coin2}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
		open_position = True
		
	if spread[-1] > up_band and not open_position:
		client.trade.place_futures_order(symbol=f"{coin1}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
		client.trade.place_futures_order(symbol=f"{coin2}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
		open_position = True

	if spread[-2] > spread_mean and spread[-1] < spread_mean and open_position:
		client.trade.close_order(symbol=f"{coin1}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
		client.trade.close_order(symbol=f"{coin2}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
		open_position = False
	
	if spread[-2] < spread_mean and spread[-1] > spread_mean  and open_position:
		client.trade.close_order(symbol=f"{coin1}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
		client.trade.close_order(symbol=f"{coin2}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
		open_position = False

	client.trade.get_positions(category="linear", settle_coin="USDT")
	print("sleeping...")
	time.sleep(20)


Calculated stats...
-0.15309098451843783
-0.053645878317945046
0.2730842221570456
-0.38037597879293567
Calculated stats...
-0.15309098451843783
-0.053645878317945046
0.2730842221570456
-0.38037597879293567


InvalidRequestError: current position is zero, cannot fix reduce-only order qty (ErrCode: 110017) (ErrTime: 18:55:01).
Request → POST https://api-testnet.bybit.com/v5/order/create: {"category": "linear", "symbol": "DOGEUSDT", "side": "Sell", "orderType": "Market", "qty": "1000", "price": null, "timeInForce": "IOC", "marketUnit": "quoteCoin", "orderLinkId": null, "reduceOnly": true}.